In [1]:
import h5py
import numpy as np
import datetime
import pandas as pd

In [2]:
# Pick which set of network Temperature you want, comment the rest
# Right now I am only using 'cent1' because the "simulations_overview" table on the MS Teams only has one
# value for each simulation

#
'''
network_Ts_sfolder = './SF16/network_Ts/'
num_sim_times = 49
startt = datetime.datetime(2019,9,28,22,0)
cp_onset = datetime.datetime(2019,9,29,5,30)
tfreq = '15min'
delta_index_hour = 5
'''
#

#

network_Ts_sfolder = './HRT12/network_Ts/'
num_sim_times = 121
startt = datetime.datetime(2019,9,29,0,0)
cp_onset = datetime.datetime(2019,9,29,5,30)
tfreq = '5min'
delta_index_hour = 13

#

In [3]:
# Hard code that there are 99 stations and 41 output timesteps along with the start time, and an array of
# "all_times". Define the "cp_onset" here as 0530Z. "delta_index_hour" used to define one hour as 5
# output timesteps

num_stations = 99
all_times = pd.date_range(startt, periods=num_sim_times, freq=tfreq)

In [4]:
'''
Loop over all the times and get all the network temperatures from that time, also save the index 
corresponding to the CP onset time and convert temperature from C to K
'''

network_Ts = np.zeros((num_sim_times,num_stations))
onset_index = np.NaN
for i,currt in enumerate(all_times):
    ct_str = currt.strftime("%Y-%m-%d-%H%M%S")
    network_T_file =  h5py.File(network_Ts_sfolder+ct_str+'.h5', 'r')
    network_Ts[i,:] = np.array(network_T_file['network_Ts'])
    network_T_file.close()
    if(currt==cp_onset):
        onset_index=i
network_Ts = network_Ts-273.15

In [5]:
'''
Determine T_ref, dT_mean, and dT_min as following from the "simulations_overview" table on MS Teams:

T_ref = Average over all stations over one hour before cold pool onset 

ΔT_mean = Mean  temperature perturbation relative to T_ref over all stations within one hour after cold 
pool onset  

ΔT_min = Strongest temperature perturbation relative to T_ref at any station over the life time of the 
cold pool 
'''

T_ref = np.mean(network_Ts[onset_index-delta_index_hour:onset_index,:])
dT_mean = np.mean(network_Ts[onset_index:onset_index+delta_index_hour,:]-T_ref)
dT_min = np.min(network_Ts[onset_index:onset_index+delta_index_hour,:]-T_ref)

In [6]:
# Print the results

print(T_ref)
print(dT_mean)
print(dT_min)

27.263672578923813
-1.5810451601225366
-3.9880159493298315


In [7]:
# Calculate the standard devation of temperature for each time

network_stds = np.std(network_Ts,axis=1)

In [8]:
'''
Determine std_mean and std_max:

std_mean = Mean standard deviation within one hour after cold pool onset

std_max = Max standard deviation over the life time of the cold pool
'''

std_mean = np.mean(network_stds[onset_index:onset_index+delta_index_hour])
std_max = np.max(network_stds[onset_index:onset_index+delta_index_hour])

In [9]:
# Print the results

print(std_mean)
print(std_max)

0.6872809735178728
1.0549883260363404
